# OpenStreetMap Data Wrangling Project


The city I chose to work with initially was Austin, Texas. 

* http://www.openstreetmap.org/relation/113314
* https://mapzen.com/data/metro-extracts/metro/austin_texas/

I chose this city because it's the place I presently call home. Tackling the Austin data set gives me a chance to get more acquainted with the place I live. 

The auditing, wrangling, and cleaning I'll set up programmatically can be applied to any area of Texas. Later in the report I'll demonstrate how the wrangling can applicable beyond the test data set.

## Problems Encountered in Data Set

As a preliminary step to working with irregularities in the data set, I'll take a look at the distribution of tags to see which are abundant enough to serve as good data wrangling practice.

In [5]:
import osm_popul_wrangler

Processing OSM file...
Data processed in 162.0023 secs. 
33 population tags found and cleaned.


Number of revised populations = 33

          Settlement  Increase  Proportion
0      Sunset Valley       -60      -0.076
1          Jonestown      -103      -0.049
2               Hays         2       0.008
3          Wimberley        42       0.016
4      Mountain City        21       0.030
5       Liberty Hill       128       0.085
6            Bastrop       687       0.091
7             Thrall        79       0.093
8         Bear Creek        41       0.109
9         San Leanna        56       0.115
10            Taylor      1895       0.124
11       Rollingwood       171       0.125
12         Woodcreek       216       0.147
13         Creedmoor        28       0.147
14            Austin    131391       0.166
15            Austin    131391       0.166
16      Pflugerville      9377       0.200
17        Round Rock     20181       0.202
18           Lakeway      2554       0.224
19     

In [4]:
# Import the libraries necessary for the project
from collections import defaultdict, Counter
import csv
import numpy as np
import re
import pprint
import sqlite3
import xml.etree.cElementTree as ET

# Open and name data file as global variable
OSM_FILE = open('austin_texas.osm', 'r', encoding="utf8")

# Element generator for parsing individual nodes in OSM file
def get_element(osm_file, tags=('node', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    __, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

# Dictionary for counting tag labels
k_tags = {}

# Population tag count
population = 0

post_codes = set([])

cities = set([])

street_desg = set([])
# Loop for going through tags
for element in get_element(OSM_FILE):
    for child in element:
        # Try-Except for dealing with tags without labels
        try:
            if child.attrib['k'] == 'addr:postcode' or child.attrib['k'] == 'tiger:zip_left':
                post_codes.add(child.get('v'))
            if child.attrib['k'] == 'addr:street':
                street = child.get('v')
                if street:
                    street_desg.add(street.split(' ')[-1])
            if child.attrib['k'] == 'addr:city':
                city = child.get('v')
                if city:
                    cities.add(city)
            if child.attrib['k'] in k_tags:
                k_tags[child.attrib['k']] += 1
            else:
                k_tags[child.attrib['k']] = 1
        except:
            continue    

# Reduce dictionaries to abundant tags
k_tags = {label:k_tags[label] for label in k_tags.keys() if k_tags[label] >= 10000}

print(post_codes)
print(street_desg)
print(cities)
print("\nAbundant tags in {}".format(OSM_FILE))
pprint.pprint(k_tags)


{'TX 78735', '78645', '78729', '78669', '78691', '78654', '78616', '78756', '78626', '78742', '76574-4649', '14150', '78676', '78758-7008', '78617', '78613', '78754', '78758-7013', '78725', '78731', '78621', '78748', '78757', '78724', 'TX 78758', '78634', '78615', '78724-1199', '78749', '78735', '78680', '78723', '78758', '78681', '78746', '78727', '78644', '78705-5609', 'TX 78724', '78759', '78610', '78719', '78728', '78732', '78612', '78957', '78730', 'TX 78613', '78652', '78745', '78664', '78663', '78738', '78712', '78737', '78726', '78641', '78628', '78750', '78751', '78705', '78640-6137', '78704', '78660', '78739', '78704-5639', '78701', '78703', '78734', '78728-1275', '78722', '78704-7205', '78721', '78717', '78753-4150', '78656', '78653', 'TX 78745', '78744', '78753', '78702', '78602', '78640', 'TX 78728', '78733', '78747', '78736', '78620', '78741', '78752', '78646', '78665', '76574'}
{'B', 'Road', '', 'Caliche', 'Tealwood', '#200N', 'Rd', '#4', 'Canyon', 'Catorce', 'cove', 'Mi

## Preliminary Audit Results

Among the most abundant node labels are address labels, which were the focus for wrangling demonstrated in P3 lessons. 

Instead of focusing on correcting address data, I'll look at data that is correctly formatted but no longer current, specifically population for the primary and satellite urban centers in the focus area. Nodes marking urban centers along with the city or town name include population and place keys for populated settlements whose values are determined by population level.



Tag|Population|Description
---|----------|-----------
place=city|100,000+|
place=town|10,000 - 100,000|an urban settlement with local importance
place=village|<10,000|incorporated municipality, regardless of its population	
place=hamlet|<100|unincorporated settlement with less than 100 inhabitants
place=isolated_dwelling|<= 2 households|the smallest kind of human settlement



In [4]:
import csv

# Create reference dictionary of 2016 estimated populations
pop_ests = {}

reader = csv.reader(open(r'2015_txpopest_place.csv'))

# Read csv and write dictionary
for row in reader:
    pop_ests[row[1]] = [row[2], row[4]] # row[1]=place name, row[2]=2010 census data, row[4]=Texas gov 2016 pop estimate
    

In [7]:
import os

# os.remove('node_attribs.csv')
os.remove('place_data.csv')
os.remove('popul_data.csv')
os.remove(r'C:\Users\User\sqlite_windows\p3_osm2')

PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\User\\sqlite_windows\\p3_osm2'

In [8]:
# Initialize csv files 
node_csv = open('node_attribs.csv', 'w')
write_node_csv = csv.writer(node_csv)

place_csv = open('place_data.csv', 'w')
write_place_csv = csv.writer(place_csv)

popul_csv = open('popul_data.csv', 'w')
write_popul_csv = csv.writer(popul_csv)

write_node_csv.writerow(('node_id', 'user', 'uid', 'timestamp'))
write_place_csv.writerow(('node_id', 'name', 'place', 'place_change'))
write_popul_csv.writerow(('name', 'osm_population', 'pop_2016', 'source'))

node_csv.close()
place_csv.close()
popul_csv.close()

def write_node_data(data):
    assert len(data) == 4
    node_csv = open('node_attribs.csv', 'a')
    writer = csv.writer(node_csv)
    writer.writerow(data)
    node_csv.close()
    
def write_place_data(data):
    assert len(data) == 4
    place_csv = open('place_data.csv', 'a')
    writer = csv.writer(place_csv)
    writer.writerow(data)
    place_csv.close()

def write_popul_data(data):
    assert len(data) == 4
    popul_csv = open('popul_data.csv', 'a')
    writer = csv.writer(popul_csv)
    writer.writerow(data)
    popul_csv.close()

    

In [9]:
places = {}

def shape_data(data):
    osm_pop = int(data[3])
    name = data[0]
    pop_2016 = int(pop_ests[name][1])
    
    if pop_2016 != osm_pop:
        data[4] = int(pop_2016)
        place = None
        if pop_2016 < 100:
            place = 'hamlet'
        elif pop_2016 <10000:
            place = 'village'
        elif pop_2016 <100000:
            place = 'town'
        else:
            place = 'city'
        if place != data[1]:
            data[1] = place
            data[2] = True
            
    else:
        data[4] = osm_pop
        
    return data

In [10]:
### Get population data from OSM file ###

#Libraries

import pprint
import xml.etree.cElementTree as ET


# Open and name data file as global variable
FILE_NAME = 'austin_texas.osm'
OSM_FILE = open(FILE_NAME, 'r', encoding="utf8")


# Element generator for parsing individual nodes in OSM file
def get_element(osm_file, tags=('node', 'way')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    __, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

for element in get_element(OSM_FILE):
    elem_id = element.get('id')
    time_stamp_year = element.get('timestamp')[:4]
    user = element.get('user')
    user_id = element.get('uid')
    node_data = [elem_id, user, user_id, time_stamp_year]
    
    name = None
    place = None
    popul = None
    source = None
    
    for tag in element.iter('tag'):
        if tag.attrib['k'] == 'name':
            name = tag.get('v')
        if tag.attrib['k'] == 'place':
            place = tag.get('v')
        if tag.attrib['k'] == 'population':
            popul = tag.get('v') 
        if tag.attrib['k'].startswith('source:population'):
            source = tag.get('v')
    
    if (name in pop_ests and popul):
        tag_data = [name, place, False, popul, None, source] 
        tag_data = [elem_id] + shape_data(tag_data)
        write_node_data(node_data)
        write_place_data(tag_data[:4])
        write_popul_data([name] + tag_data[4:])
    

In [10]:
connect = sqlite3.connect(r'C:\Users\User\sqlite_windows\p3_osm2')
cur = connect.cursor()
cur.execute('.tables')

OperationalError: near ".": syntax error

In [11]:
import csv, sqlite3
import pandas as pd



nodes_df = pd.read_csv("node_attribs.csv")
places_df = pd.read_csv("place_data.csv")
popul_df = pd.read_csv("popul_data.csv")

connect = sqlite3.connect(r'C:\Users\User\sqlite_windows\p3_osm2')

cur = connect.cursor()
cur.execute('DROP TABLE IF EXISTS settlement_nodes;')
cur.execute('DROP TABLE IF EXISTS settlement_places;')
cur.execute('DROP TABLE IF EXISTS settlement_popul;')

nodes_df.to_sql('settlement_nodes', connect)
places_df.to_sql('settlement_places', connect)
popul_df.to_sql('settlement_popul', connect)
# Initialize tables



# cur = connect.cursor()
# cur.execute("CREATE TABLE settlement_nodes (node_id integer PRIMARY KEY,\
#                                             user text,\
#                                             uid integer,\
#                                             timestamp integer);")

# cur.execute("CREATE TABLE settlement_tags (node_id integer,\
#                                            name text,\
#                                            place text,\
#                                            place_change boolean,\
#                                            osm_pop integer,\
#                                            pop_2016,\
#                                            source);")

# def csv_pull(data_file):
#     to_db = []
#     d = csv.DictReader(open(data_file))
#     col_names = d.fieldnames
#     for row in d:
#         temp = tuple((row[col] for col in col_names))
#         to_db.append(temp)
#     return to_db
    
# node_to_db = csv_pull()

# curs.executemany("INSERT INTO TX_popul_est (place, popul_2016) VALUES (?, ?);", to_db)
# curs.execute("CREATE TABLE IF NOT EXISTS OSM_Pop_Data_Nodes(node_id integer PRIMARY KEY, place text, designation text, popul integer, FOREIGN KEY(place) REFERENCES TX_popul_est(place));")
# curs.executemany("INSERT INTO OSM_Pop_Data_Nodes (node_id, place, designation, popul) VALUES(?, ?, ?, ?);", names)
# p3_db.commit()
# p3_db.close()

AttributeError: 'sqlite3.Cursor' object has no attribute 'tables'

In [52]:
import sqlite3
import pandas as pd
import pprint

p3_db = sqlite3.connect(r'C:\Users\User\sqlite_windows\p3_osm2')
curs = p3_db.cursor()
curs.execute("SELECT COUNT(name) FROM settlement_places;")
n = curs.fetchone()[0]
print("Number of revised populations = {}".format(n))
print()
curs.execute("SELECT name, pop_2016 - osm_population,\
                     round(((pop_2016 - osm_population)/(osm_population*1.0)), 3) AS proportion\
                     FROM settlement_popul \
                     ORDER BY proportion;")
settlements = curs.fetchall()
settlements_df = pd.DataFrame(settlements, columns=('Settlement', 'Increase', 'Proportion'))
print(settlements_df)
print()
curs.execute("SELECT round(avg(pop_2016 - osm_population), 2) AS average_increase,\
                     avg(1.0 * round(((pop_2016 - osm_population)/(1.0 * osm_population)), 2)) AS proportion\
                     FROM settlement_popul;")
incr_avg, prop_avg = curs.fetchone()
print("Mean population increase = {},  Mean proportional increase = {}".format(incr_avg, prop_avg))
print()
curs.execute("SELECT sum(place_change) AS Place_Changes FROM settlement_places;")
n_place_change = curs.fetchone()[0]

print("Number of place designations changed = {}".format(n_place_change))
print()
curs.execute("SELECT name, place FROM settlement_places WHERE place_change == 1")
place_change = curs.fetchall()
place_change_df = pd.DataFrame(place_change, columns=('Settlement', 'New Designation'))
print(place_change_df)
print()



Number of revised populations = 28

          Settlement  Increase  Proportion
0      Sunset Valley       -60      -0.076
1          Jonestown      -103      -0.049
2               Hays         2       0.008
3          Wimberley        42       0.016
4      Mountain City        21       0.030
5       Liberty Hill       128       0.085
6            Bastrop       687       0.091
7             Thrall        79       0.093
8         Bear Creek        41       0.109
9         San Leanna        56       0.115
10            Taylor      1895       0.124
11       Rollingwood       171       0.125
12         Woodcreek       216       0.147
13         Creedmoor        28       0.147
14            Austin    131391       0.166
15      Pflugerville      9377       0.200
16        Round Rock     20181       0.202
17           Lakeway      2554       0.224
18              Kyle     10309       0.368
19       Webberville       120       0.390
20        Georgetown     22009       0.518
21           Volen